# 1. Correcting for magnetic declination and horizontal movement

Input is a the DCPS.txt file from the datalogger, output is a fully annotated NETCDF-file, with data corrected for magnetic declination and horizontal movement.

Written by Johan Edholm, 5 July 2022

In [1]:
# Load modules and make some functions

import xarray as xr
import pandas as pd
import numpy as np
import datetime
import geomag
import gsw
import glidertools as gt
from tqdm.notebook import tqdm_notebook as tqdm

def get_bearing(lat,lon):
    
    lat1 = np.deg2rad(lat[:-1])
    lat2 = np.deg2rad(lat[1:])
    lon1 = np.deg2rad(lon[:-1])
    lon2 = np.deg2rad(lon[1:])

    dLon = lon2 - lon1;
    y = np.sin(dLon) * np.cos(lat2);
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) *np.cos(lat2) * np.cos(dLon);
    brng = ((np.rad2deg(np.arctan2(y, x)) + 360) % 360)

    return brng

## Load and correct the DCPS currents

The .txt-file looks like this:
***
Sensorlog opened 10.01.2022 12:05:26 GPS: 10.01.2022 12:05:26 -63.29883 2.44779


%

StartupInfo	5400	472	Mode	AADI Smart Sensor Terminal Protocol	RS232 Protocol Version	3	Config Version	49

Initializing...

Started...

MEASUREMENT	5400	472	Heading[Deg.M]	9.633996E+01 ...

In [12]:
#---- Load the datafile ----#
print('Loading...')
df = pd.read_csv('/Users/thedon/Dropbox/Work/SailBuoy Internship/Data/2022_SOCHIC_Maudrise/DCPS.TXT',names=['1']) # Unchanged DCPS.txt file from the datalogger.

#---- Select only the DCPS output and split after each tab ----#
print('Selecting...')
df1 = df.loc[np.where(df['1'].str[:3] == '***')[0]-1]['1'].str.split('	',expand=True)# Select only the rows that contain MEASUREMENT (which is the start of the DCPS output), and then split the resulting cell for every \t adding new columns accordingly.

#---- Select every other cell, and set that as the next columns name ----#
n = df1.shape[1]-1
columns = df1.iloc[0].iloc[np.arange(3,n,2)].values # Get the column names that matter
df1 = df1.drop(np.arange(3,n,2),axis=1).drop([0,1,2],axis=1) # First, drop all the columns with just the names. Second, drop the three first columns, since they stay the same for all rows.
df1.columns = columns # Replace the column names with the proper ones

#---- Split between what varies with only time and cell ----#
df2 = df1.iloc[:,:16] # Varies only with time
df3 = df1.iloc[:,16:] # Varies with time AND cell

#---- Make a container for each cell ----#
tmp = []
for i in range(40):
    tmp.append(df1.iloc[:,16+i*int(df3.shape[1]/40):16+int(df3.shape[1]/40)+i*int(df3.shape[1]/40)])

#---- Make new datasets that varies with time, and time and cell ----#
print()
DS = xr.Dataset(tmp[0]).expand_dims('cell').assign_coords(cell=('cell',[0]))
for i in tqdm(range(39),'Creating new datasets'):
    DS = xr.concat([DS,xr.Dataset(tmp[i+1]).expand_dims('cell').assign_coords(cell=('cell',[i+1]))],dim='cell')
DS = DS.rename({'dim_0':'index'})
ds = df2.to_xarray()

#---- Merge the two datasets ----#
print('Merging...')
dcps = xr.merge([ds.astype(float),DS.astype(float)])

#---- Rename variables and set units ----#
keys = list(dcps.keys()) # Array with all the variables. Some of them have the units in the variable name, and we'd like to move this to the attributes instead.
start = [] # Placeholder
end = [] # Placeholder
for j in range(len(keys)): # For each variable
    for i in range(len(list(keys[j]))): # For the length of the variable name
        if (list(keys)[j][i] == '[') == 1: # If there is a [, then save the j (variable index in keys) and i (character index in variable name).
            start.append([j,i])
        if (list(keys)[j][i] == ']') == 1: # If there is a ], then save the j (variable index in keys) and i (character index in variable name). Length of the units can vary, hence we need this too.
            end.append([j,i])

for i in tqdm(range(len(start)),'Renaming the variables'):
        dcps[keys[start[i][0]]].attrs['Units'] = keys[start[i][0]][start[i][1]+1:end[i][1]] # Add the units from what's in the variable name
        dcps = dcps.rename({keys[start[i][0]]:keys[start[i][0]][:start[i][1]]})             # Remove the units from the variable name


idx_open = df.iloc[df['1'].str.contains('opened').values].index.values # Select all rows that contain "openened". The sensor always log when it turns on.
idx_close = df.iloc[df['1'].str.contains('closed').values].index.values # Select all rows that contain "closed". The sensor always log when it turns off.

idx = np.where(idx_close-idx_open == 47)[0] # Make sure that we only select indices that coincides with a measurement.
idx_open = idx_open[idx]
idx_close = idx_close[idx]

good_idx = []
for i in tqdm(range(len(idx_open)),'Removing bad indices'):
    if ((idx_open[i] == df1.index.values - 5) & (idx_close[i] == df1.index.values + 42)).sum() == 1:
        good_idx.append(i)

idx_open = idx_open[good_idx]
idx_close = idx_close[good_idx]

lat = np.zeros(len(idx_open))
lon = np.zeros(len(idx_open))
sample_time = np.zeros(len(idx_open))
sample_dist = np.zeros(len(idx_open))
sample_vel = np.zeros(len(idx_open))
heading = np.zeros(len(idx_open))
northing = np.zeros(len(idx_open))
easting = np.zeros(len(idx_open))

ii = []
dcps['index'] = dcps['index'].astype('datetime64[s]') # It didn't like assigning datetime.datetime without this
for i in tqdm(range(len(idx_open)),'Fixing the time'): 
    t_open = datetime.datetime(int(df['1'].values[idx_open[i]][23:27]), # Year: when the sensor opens
                               int(df['1'].values[idx_open[i]][20:22]), # Month: -
                               int(df['1'].values[idx_open[i]][17:19]), # Day: -
                               int(df['1'].values[idx_open[i]][28:30]), # Hour: -
                               int(df['1'].values[idx_open[i]][31:33]), # Minute: -
                               int(df['1'].values[idx_open[i]][34:36])) # Second: -
    t_close = datetime.datetime(int(df['1'].values[idx_close[i]][23:27]), # Year: when the sensor closed
                                int(df['1'].values[idx_close[i]][20:22]), # Month: -
                                int(df['1'].values[idx_close[i]][17:19]), # Day: -
                                int(df['1'].values[idx_close[i]][28:30]), # Hour: -
                                int(df['1'].values[idx_close[i]][31:33]), # Minute: -
                                int(df['1'].values[idx_close[i]][34:36])) # Second: -
    dcps['index'].values[i] = t_open # Set the values to when the sensor opened. MUST check this with other sensors, which one to choose.
    if (len(df.iloc[idx_close[i]][0][71:79]) > 0) & (len(df.iloc[idx_open[i]][0][62:70]) > 0): # This is to determine whether or not the sensor got a GPS fix
        ii += [i]
        lats = float(df.iloc[idx_open[i]][0][62:70]), float(df.iloc[idx_close[i]][0][62:70])
        lons = float(df.iloc[idx_open[i]][0][71:79]), float(df.iloc[idx_close[i]][0][71:79])
        sample_time[i] = (t_close - t_open).total_seconds()   
        sample_dist[i] = gt.utils.distance(lons,lats)[-1]
        sample_vel[i] = sample_dist[i]/sample_time[i]
        lat[i] = np.nanmean(lats)
        lon[i] = np.nanmean(lons)
        heading[i] = get_bearing(lats,lons)
        northing[i] = ((lats[1]-lats[0]) * 60 * 1852)/sample_time[i]
        easting[i] = ((lons[1]-lons[0]) * np.cos(np.deg2rad(lats[1])) * 60 * 1852)/sample_time[i]
        
dcps = dcps.rename({'index':'time'})
dcps = dcps.isel(time=ii)
dcps = dcps.assign_coords(latitude=('time', lat[ii])).assign_coords(longitude=('time', lon[ii])).assign_coords(sample_time=('time', sample_time[ii]))
dcps = dcps.assign_coords(sample_distance=('time', sample_dist[ii])).assign_coords(sample_velocity=('time', sample_vel[ii])).assign_coords(sample_heading=('time', heading[ii]))
dcps = dcps.assign_coords(northing=('time', northing[ii])).assign_coords(easting=('time', easting[ii]))

dcps['time'] = dcps.time.to_pandas().round('15min').values # Round down to the nearest quarter
dcps = dcps.assign_coords(depth=('cell',np.arange(3,83,2))) # Make sure that we have the correct depth values for the cells. Blanking distance is 3 m.
mag_dec = np.zeros(len(dcps.time))
for i in range(len(dcps.time)):
    mag_dec[i] = geomag.declination(dcps.latitude[i].data,dcps.longitude[i].data)
dcps['mag_dec'] = xr.DataArray(mag_dec,dims={'time'},coords={'time':dcps.time})


u = dcps['Horizontal Speed'] * np.sin(np.deg2rad(dcps['Direction']+dcps['mag_dec'])) / 100  # DCPS east velocity
v = dcps['Horizontal Speed'] * np.cos(np.deg2rad(dcps['Direction']+dcps['mag_dec'])) / 100 # DCPS north velocity

sb_u = dcps['easting'] # SB east velocity
sb_v = dcps['northing'] # SB north velocity
# Easting and Northing are calculated from the GPS positions when the sensor open/closes, and the time stamps related to that.

corr_u = u + sb_u # DCPS east velocity corrected for magnetic declination and the SB east velocity 
corr_v = v + sb_v # DCPS north velocity corrected for magnetic declination and the SB north velocity 

direction = np.rad2deg(np.arctan2(corr_u,corr_v)) % 360 # Actual current direction, refereced from the north

dcps['North Speed'] = corr_v
dcps['East Speed'] = corr_u
dcps['Horizontal Speed'] = (corr_u**2 + corr_v**2)**(1/2)
dcps['Direction'] = direction
dcps = dcps.assign_coords(sb_u=('time',sb_u.values)).assign_coords(sb_v=('time',sb_v.values))

print('Saving...')
#dcps.to_netcdf('/Users/thedon/Dropbox/Work/SailBuoy Internship/Data/2022_SOCHIC_Maudrise/DCPS_corrected.nc')
print('Done!')

Loading...
Selecting...
4 rows dropped.



Creating new datasets:   0%|          | 0/39 [00:00<?, ?it/s]

Merging...


Renaming the variables:   0%|          | 0/34 [00:00<?, ?it/s]

Removing bad indices:   0%|          | 0/6115 [00:00<?, ?it/s]

Fixing the time:   0%|          | 0/6114 [00:00<?, ?it/s]

Saving...
Done!


## Update the names, units, and description for the variables

In [13]:
standard_names = ['heading',
                  'sd_heading',
                  'pitch',
                  'roll',
                  'abs_tilt',
                  'max_tilt',
                  'sd_tilt',
                  'tilt_direction',
                  'charge_voltage_vtx1',
                  'charge_voltage_vtx2',
                  'min_input_voltage',
                  'input_voltage',
                  'input_current',
                  'memory_used',
                  'record_state',
                  'ping_count',
                  'cell_index',
                  'cell_state1',
                  'cell_state2',
                  'horizontal_speed',
                  'direction',
                  'north_speed',
                  'east_speed',
                  'vertical_speed',
                  'sp_sd_horizontal',
                  'strength',
                  'beam1_speed',
                  'beam2_speed',
                  'beam3_speed',
                  'beam4_speed',
                  'beam1_strength',
                  'beam2_strength',
                  'beam3_strength',
                  'beam4_strength',
                  'beam1_sd',
                  'beam2_sd',
                  'beam3_sd',
                  'beam4_sd',
                  'cross_difference',
                  'mag_dec']

units = ['degrees.M',
         'degrees.M',
         'degrees',
         'degrees',
         'degrees',
         'degrees',
         'degrees',
         'degrees.M',
         'V',
         'V',
         'V',
         'V',
         'mA',
         'bytes',
         '',
         '',
         '',
         '',
         '',
         'm/s',
         'degrees.M',
         'm/s',
         'm/s',
         'm/s',
         'm/s',
         'strength',
         'm/s',
         'm/s',
         'm/s',
         'm/s',
         'dB',
         'dB',
         'dB',
         'dB',
         'm/s',
         'm/s',
         'm/s',
         'm/s',
         'm/s',
         'degrees']

long_names = ['Heading','Standard deviation heading',
              'Pitch',
              'Roll',
              'Absolute tilt',
              'Maximum tilt',
              'Standard deviation tilt',
              'Tilt direction',
              'Charge voltage Vtx1',
              'Charge voltage vtx2',
              'Minimum input voltage',
              'Input voltage',
              'Input current',
              'Memory used',
              'Record state',
              'Ping count',
              'Cell index',
              'Cell state 1',
              'Cell state2',
              'Horizontal speed',
              'Direction',
              'North speed',
              'East speed',
              'Vertical speed',
              'Single ping standard deviation horizontal speed',
              'Strength',
              'Beam 1 speed',
              'Beam 2 speed',
              'Beam 3 speed',
              'Beam 4 speed',
              'Beam 1 strength',
              'Beam 2 strength',
              'Beam 3 strength',
              'Beam 4 strength',
              'Beam 1 standard deviation',
              'Beam 2 standard deviation',
              'Beam 3 standard deviation',
              'Beam 4 standard deviation',
              'Cross difference',
              'Magnetic declination']

description = ['Averaged heading from one interval, one heading measurement per ping, vector averaged.',
               'Standard deviation calculation on all heading values from one interval. Indicates how much the sensor rotates around the vertical axis during a measurement interval.',
               'Pitch angle, average from one interval, one tilt measurement per ping. Pitch is the rotation angle around the x-axis of the sensor (same axis as Transducer 1 and 3).',
               'Roll angle, average from one interval, one tilt measurement per ping. Roll is the rotation angle around the y-axis of the sensor (same axis as transducer 4 and 2).',
               'Angle between sensor plane and horizontal plane. Calculates one value per ping from pitch and roll angles, average of all values.',
               'Maximum absolute tilt from the interval',
               'Standard deviation tilt from all values of the absolute tilt in the interval. Indicates if the sensor is moving around with variable tilt during the measurement interval.',
               'A tilt direction is calculated per ping, this is the average from the interval. Gives the direction where the sensor has its largest tilt, with magnetic north as reference.',
               'The measured voltage to the capacitor on transmitter electronics to Transducer 1 and 2. It should normally be > 4.8V.',
               'The measured voltage to the capacitor on transmitter electronics to Transducer 3 and 4. It should normally be > 4.8V.',
               'The minimum input voltage measured while charging the capacitor bank. It should normally be > 6.0V.',
               'The voltage measured when not charging while awake, averaged.',
               'The current measured when not charging while awake, averaged.',
               'Used heap memory.',
               'A 32-bit status number which provides quality warnings.',
               'Number of pings executed, can be lower than configured number of pings to be done.',
               'An index which gives column and cell number.  A 1xxx is column 1, 2xxx is column 2 and 3xxx is column 3.',
               'The cell state 1 indicates different conditions like for example if the cell has weak signal or cell is inside blanking zone or illegible zone. If zero, everything is ok.',
               'The cell state 2 is an addition to cell state 1 and indicates different conditions on beam level like for example beam 1, beam 2, beam 3 or beam 4 is inside blanking zone or illegible zone. If zero, everything is ok.',
               'Horizontal speed calculated from the 4 beams and compensated for tilt, average of all pings over the interval duration.',
               'The current direction calculated from the 4 beams combined with compass heading to determine the current direction.',
               'North speed component, average of all pings over the recording interval.',
               'East speed component, average of all pings over the recording interval.',
               'Vertical speed component, average of all pings over the recording interval.',
               'Single ping standard deviation is the standard deviation calculated from all horizontal speed data in the interval. Divide by the square root of number of pings to get the standard deviation for the set.',
               'Averaged signal from the four beams. The signal strength is calculated for each beam for each ping and averaged at the end of the interval.',
               'Averaged speed for beam 1.',
               'Averaged speed for beam 2.',
               'Averaged speed for beam 3.',
               'Averaged speed for beam 4.',
               'Averaged strength for beam 1.',
               'Averaged strength for beam 2.',
               'Averaged strength for beam 3.',
               'Averaged strength for beam 4.',
               'Standard deviation calculated from all the beam 1 speeds in the interval.',
               'Standard deviation calculated from all the beam 2 speeds in the interval.',
               'Standard deviation calculated from all the beam 3 speeds in the interval.',
               'Standard deviation calculated from all the beam 4 speeds in the interval.',
               'Calculated as (Beam 1 Speed + Beam 3 Speed) - (Beam 2 Speed + Beam 4 Speed), where the BeamX speeds are tilt compensated. The Cross Difference is calculated for each ping and averaged at the end of the interval. In a homogeneous water flow this value is close to zero.',
               'Local magnetic declination, determined by the longitude and latitude.']


In [14]:
c = 0
for var in list(dcps.keys()):
    dcps[var].attrs['standard_name'] = standard_names[c]
    dcps[var].attrs['long_name'] = long_names[c]
    dcps[var].attrs['units'] = units[c]
    dcps[var].attrs['description'] = description[c]
    dcps = dcps.rename({var:standard_names[c]})
    c += 1

In [16]:
dcps.drop(['sample_time',
 'sample_distance',
 'sample_velocity',
 'sample_heading',
 'northing',
 'easting',
 'sb_u',
 'sb_v']).to_netcdf('/Users/thedon/Dropbox/Work/SailBuoy Internship/Data/2022_SOCHIC_Maudrise/DCPS_corrected.nc')

## Done!